In [2]:
const ○ = 1 # 𝟘 ○ OFF
const ● = 2 # 𝟙 ● ON

2

In [3]:
state_count = 4

# state by state flat 
p = 
Float64[
#              ○ ● ○ ● ○ ● ○ ● ○ ● ○ ● ○ ● ○ ●
#              ○ ○ ● ● ○ ○ ● ● ○ ○ ● ● ○ ○ ● ●
#              ○ ○ ○ ○ ● ● ● ● ○ ○ ○ ○ ● ● ● ●
#              ○ ○ ○ ○ ○ ○ ○ ○ ● ● ● ● ● ● ● ●    
#= ○ ○ ○ ○ =#  1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ● ○ ○ ○ =#  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ● ○ ○ =#  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0;
#= ● ● ○ ○ =#  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ○ ● ○ =#  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0;
#= ● ○ ● ○ =#  0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0;
#= ○ ● ● ○ =#  0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0;
#= ● ● ● ○ =#  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0;
#= ○ ○ ○ ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ● ○ ○ ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ○ ● ○ ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ● ● ○ ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ○ ○ ● ● =#  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0;
#= ● ○ ● ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1;
#= ○ ● ● ● =#  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0;
#= ● ● ● ● =#  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0;
]
q = reshape(p, (ntuple(x->2, state_count*2)));
length(size(q))

8

In [20]:
mutable struct TPM
    tpm
    current_state_elements # element -> tpm dimension
    next_state_elements # element -> tpm dimension
end


function make_tpm(flat, elements)
    @assert size(flat) == (length(elements)^2, length(elements)^2)
    
    TPM(
        reshape(flat, (ntuple(x->2, length(elements)*2))),
        Dict([(value, index) for (index, value) in enumerate(elements)]),
        Dict([(value, index + length(elements)) for (index, value) in enumerate(elements)])
    )
end

function adjust_dims(elements, removed_dim)
    Dict([dim > removed_dim ? (element, dim-1) : (element, dim) for (element, dim) in elements])
end

function flatten_tpm(tpm)
    reshape(tpm.tpm, 2^length(tpm.current_state_elements), 2^length(tpm.next_state_elements)) 
end

function fix_condition(tpm, element, current, value) 
    @assert value == ○ || value == ●
    
    len = length(size(tpm.tpm))
    
    dim = current ? tpm.current_state_elements[element] : tpm.next_state_elements[element]
    tpm2 = tpm.tpm[ntuple(x->Colon(), dim-1)..., value, ntuple(x->Colon(), len-dim)...]
    
    current_state_elements = adjust_dims(tpm.current_state_elements, dim)
    next_state_elements = adjust_dims(tpm.next_state_elements, dim)

    delete!(current ? current_state_elements : next_state_elements, element)
    
    TPM(tpm2, current_state_elements, next_state_elements)
end 

function marginalize(tpm, element, current) 
    println("marginalize", size(tpm.tpm), element, current ? "Current" : "Next")
    len = length(size(tpm.tpm))
    
    dim = current ? tpm.current_state_elements[element] : tpm.next_state_elements[element]
    
    current_count = length(tpm.current_state_elements)
   
    tpm2 = tpm.tpm[ntuple(x->Colon(), dim-1)..., ○, ntuple(x->Colon(), len-dim)...] + tpm.tpm[ntuple(x->Colon(), dim-1)..., ●, ntuple(x->Colon(), len-dim)...]
    tpm3 = tpm2 ./ sum(tpm2, dims=(current ? [current_count:len-1...] : [current_count+1:len-1...])) # normailizing rows
    
    current_state_elements = adjust_dims(tpm.current_state_elements, dim)
    next_state_elements = adjust_dims(tpm.next_state_elements, dim)
    
    delete!(current ? current_state_elements : next_state_elements, element)

    TPM(tpm3, current_state_elements, next_state_elements)
end 

marginalize (generic function with 1 method)

In [72]:
struct System
    elements
    states
    element_states
end

function make_system(elements, states)
    System(elements, states, Dict([(elements[index], states[index]) for (index, element) in enumerate(elements)]))
end


struct CandidateMechanism
    elements
    mechanism_elements
    perview_elements
end

function system_elements(system)
    keys(system.element_states)
end

function external_elements(system, mechanism)
    setdiff(system.elements, mechanism.elements)
end

function fix_elements(tpm, system, elements)
    for element in elements
        tpm = fix_condition(tpm, element, true, system.element_states[element])
        tpm = marginalize(tpm, element, false) 
    end
    tpm
end


function foo(tpm)
    x = Vector(undef, length(tpm.current_state_elements))
    for (element, dim) in tpm.current_state_elements
        x[dim] = element
    end
    
    y = Vector(undef, length(tpm.next_state_elements))
    for (element, dim) in tpm.next_state_elements
        y[dim-length(tpm.current_state_elements)] = element
    end
    
    (x,y)
end


# Cause Repertoire
# Marganilize out non perview elements
# Marganilize out non mechanism elements

# Effect Repertoire
# Marganilize out non perview elements
# Marganilize out non mechanism elements
    
function repertoire(tpm, system, mechanism, cause)
    # effect repertoire these are t+1
    # cause repertoire these are t
    for element in setdiff(mechanism.elements, mechanism.perview_elements)
        tpm = marginalize(tpm, element, cause)
    end
    
    # effect repertoire these are t
    # cause repertoire these are t+1
    for element in setdiff(mechanism.elements, mechanism.mechanism_elements)
        tpm = marginalize(tpm, element, !cause)
    end
    

    x, y = foo(tpm)
    
    for cause || in(element, mechanism.mechanism_elements) for element in x
        adjust_dims(tpm.current_state_elements, tpm.current_state_elements[removed_dim])
    end
    
    for !cause || in(element, mechanism.mechanism_elements) for element in y
        
    end
    
    tpm2 = tpm.tpm[
        [!cause && in(element, mechanism.mechanism_elements) ? system.element_states[element] : Colon() for element in x]...,
        [cause && in(element, mechanism.mechanism_elements) ? system.element_states[element] : Colon() for element in y]...
    ]
    
    TPM(tpm2, current_state_elements, next_state_elements)
end

function cause_repertoire(tpm, system, mechanism)
    repertoire(tpm, system, mechanism, true)
end

function effect_repertoire(tpm, system, mechanism)
    repertoire(tpm, system, mechanism, false)
end


system = make_system([:a, :b, :c, :d], [●, ○, ○, ○])

println(system)

tpm = make_tpm(p, system.elements)

mechanism = CandidateMechanism([:a, :b, :c], [:c], [:b, :c])

tpm = fix_elements(tpm, system, external_elements(system, mechanism))

println(flatten_tpm(tpm))

println("cause_repertoire")
tpm_cause = cause_repertoire(tpm, system, mechanism)
println("effect_repertoire")
tpm_effect = effect_repertoire(tpm, system, mechanism)

println(tpm_cause)
println(tpm_effect)

println(flatten_tpm(tpm_cause))
println(flatten_tpm(tpm_effect))

# tpm_a = fix_condition(tpm_a, :d, true, ○)
# tpm_a = marginalize(tpm_a, :d, false)

# tpm_b = marginalize(tpm_a, :b, false)
# tpm_b = marginalize(tpm_b, :c, false)
# tpm_b = marginalize(tpm_b, :a, true)
# tpm_b = marginalize(tpm_b, :b, true)
# tpm_b = marginalize(tpm_b, :c, true)

# tpm_c = marginalize(tpm_a, :a, true)
# tpm_c = marginalize(tpm_c, :b, true)

# tpm_d = marginalize(tpm_c, :a, false)
# tpm_d = fix_condition(tpm_d, :c, true, ○)

# tpm_e = marginalize(tpm_c, :b, false)
# tpm_e = fix_condition(tpm_e, :c, true, ○)

# println(flatten_tpm(tpm_d))
# println(flatten_tpm(tpm_e))

System([:a, :b, :c, :d], [2, 1, 1, 1], Dict(:a => 2, :b => 1, :d => 1, :c => 1))
marginalize(2, 2, 2, 2, 2, 2, 2)dNext
[1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0]
cause_repertoire
marginalize(2, 2, 2, 2, 2, 2)aCurrent
marginalize(2, 2, 2, 2, 2)aNext
marginalize(2, 2, 2, 2)bNext
effect_repertoire
marginalize(2, 2, 2, 2, 2, 2)aNext
marginalize(2, 2, 2, 2, 2)aCurrent
marginalize(2, 2, 2, 2)bCurrent
TPM([0.5 0.5; 0.5 0.5], Dict(:b => 1, :c => 2), Dict(:c => 3))
TPM([0.5 0.5; 0.0 0.0], Dict(:c => 1), Dict(:b => 2, :c => 3))


LoadError: DimensionMismatch("new dimensions (4, 2) must be consistent with array size 4")

In [6]:
el = Dict(:a => ●, :b => ○, :c => ○, :d => ○)

Dict{Symbol, Int64} with 4 entries:
  :a => 2
  :b => 1
  :d => 1
  :c => 1

In [12]:
[keys(el)...]

4-element Vector{Symbol}:
 :a
 :b
 :d
 :c

In [10]:
?KeySet

search:

Couldn't find KeySet
Perhaps you meant BitSet, Set, reset or keys


No documentation found.

Binding `KeySet` does not exist.


In [17]:
?contains

search: contains



```
contains(haystack::AbstractString, needle)
```

Return `true` if `haystack` contains `needle`. This is the same as `occursin(needle, haystack)`, but is provided for consistency with `startswith(haystack, needle)` and `endswith(haystack, needle)`.

See also [`occursin`](@ref), [`in`](@ref), [`issubset`](@ref).

# Examples

```jldoctest
julia> contains("JuliaLang is pretty cool!", "Julia")
true

julia> contains("JuliaLang is pretty cool!", 'a')
true

julia> contains("aba", r"a.a")
true

julia> contains("abba", r"a.a")
false
```

!!! compat "Julia 1.5"
    The `contains` function requires at least Julia 1.5.


---

```
contains(needle)
```

Create a function that checks whether its argument contains `needle`, i.e. a function equivalent to `haystack -> contains(haystack, needle)`.

The returned function is of type `Base.Fix2{typeof(contains)}`, which can be used to implement specialized methods.


In [39]:
?Vector

search: Vector BitVector DenseVector StridedVector AbstractVector



```
Vector{T} <: AbstractVector{T}
```

One-dimensional dense array with elements of type `T`, often used to represent a mathematical vector. Alias for [`Array{T,1}`](@ref).

See also [`empty`](@ref), [`similar`](@ref) and [`zero`](@ref) for creating vectors.

---

```
Vector{T}(undef, n)
```

Construct an uninitialized [`Vector{T}`](@ref) of length `n`.

# Examples

```julia-repl
julia> Vector{Float64}(undef, 3)
3-element Array{Float64, 1}:
 6.90966e-310
 6.90966e-310
 6.90966e-310
```

---

```
Vector{T}(nothing, m)
```

Construct a [`Vector{T}`](@ref) of length `m`, initialized with [`nothing`](@ref) entries. Element type `T` must be able to hold these values, i.e. `Nothing <: T`.

# Examples

```jldoctest
julia> Vector{Union{Nothing, String}}(nothing, 2)
2-element Vector{Union{Nothing, String}}:
 nothing
 nothing
```

---

```
Vector{T}(missing, m)
```

Construct a [`Vector{T}`](@ref) of length `m`, initialized with [`missing`](@ref) entries. Element type `T` must be able to hold these values, i.e. `Missing <: T`.

# Examples

```jldoctest
julia> Vector{Union{Missing, String}}(missing, 2)
2-element Vector{Union{Missing, String}}:
 missing
 missing
```


In [64]:
?popat!

search: popat! DEPOT_PATH



```
popat!(a::Vector, i::Integer, [default])
```

Remove the item at the given `i` and return it. Subsequent items are shifted to fill the resulting gap. When `i` is not a valid index for `a`, return `default`, or throw an error if `default` is not specified.

See also: [`pop!`](@ref), [`popfirst!`](@ref), [`deleteat!`](@ref), [`splice!`](@ref).

!!! compat "Julia 1.5"
    This function is available as of Julia 1.5.


# Examples

```jldoctest
julia> a = [4, 3, 2, 1]; popat!(a, 2)
3

julia> a
3-element Vector{Int64}:
 4
 2
 1

julia> popat!(a, 4, missing)
missing

julia> popat!(a, 4)
ERROR: BoundsError: attempt to access 3-element Vector{Int64} at index [4]
[...]
```


In [65]:
[1:10]

1-element Vector{UnitRange{Int64}}:
 1:10

In [66]:
[1:10...][1:5 
    7:10]

LoadError: MethodError: no method matching typed_hcat(::Vector{Int64}, ::UnitRange{Int64}, ::UnitRange{Int64})
[0mClosest candidates are:
[0m  typed_hcat([91m::Type{T}[39m, ::AbstractVecOrMat...) where T at /opt/julia-1.7.1/share/julia/base/abstractarray.jl:1557
[0m  typed_hcat([91m::Type[39m, ::AbstractArray, ::AbstractArray) at /opt/julia-1.7.1/share/julia/base/abstractarray.jl:1877
[0m  typed_hcat([91m::Type[39m, ::AbstractArray...) at /opt/julia-1.7.1/share/julia/base/abstractarray.jl:1878
[0m  ...